Importing required libraries

In [26]:
import xml.etree.ElementTree as ET
import os
from pathlib import Path
import csv


Function to extract footnotes and save them to a CSV file.

In [43]:
# Define the namespace used in the XML
namespace = {'pc': 'http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15'}

# Find all TextRegion elements with 'footnote' in their custom attribute
footnote_data = {}
field_names = ['File Name', 'Footnote']
def extract_footnotes(filepath):
    xml_files_path = Path(filepath).rglob('*.xml')
    for file in xml_files_path:
        xml_file = str(file)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for text_region in root.findall(".//pc:TextRegion", namespace):
            page = root.find('.//pc:Page', namespace)
            page_name = page.attrib.get('imageFilename')
            custom_attr = text_region.attrib.get('custom', '')
            if 'structure {type:footnote;}' in custom_attr:
                footnotes_text = []
                current_footnote = []
                # Extract TextLine elements within the TextRegion
                for text_line in text_region.findall(".//pc:TextLine", namespace):
                    # Get the Unicode content
                    unicode_elem = text_line.find(".//pc:Unicode", namespace)
                    if unicode_elem is not None:
                        text = unicode_elem.text.strip()
                        # Check if it is a new footnote
                        if text.startswith('*') or text.startswith('†') or text.startswith('‡'):
                            # Save any existing footnote
                            if current_footnote:
                                footnotes_text.append(' '.join(current_footnote))
                                # Clear out current footnote data
                                current_footnote = []
                            # Update current_footnote with new footnote text
                            current_footnote.append(text)
                        # If it not a new footnote, continue adding to existing footnote text
                        else:
                            current_footnote.append(text)
                # Add last footnote
                if current_footnote:
                    footnotes_text.append(' '.join(current_footnote))
                # Save footnotes information to dictionary
                footnote_data.update({
                    page_name: footnotes_text})
    
    # Write the footnotes to a CSV file
    output_file = Path(filepath) / 'extracted_footnotes.csv'
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(field_names)

        for file_name, footnotes in footnote_data.items():
            for footnote in footnotes:
                writer.writerow([file_name, footnote])
                
    print(f"Footnotes extracted and saved to {output_file}")

extract_footnotes('data/footnotes/the_land_of_the_perumauls/files/The_Land_of_the_Perumauls/page')
# print(len(footnote_data))
# print((footnote_data['0064_the_land_of_the_perumauls_page_64.jpg']))

Footnotes extracted and saved to data/footnotes/the_land_of_the_perumauls/files/The_Land_of_the_Perumauls/page/extracted_footnotes.csv
